In [1]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

     |████████████████████████████████| 61kB 4.1MB/s 
     |████████████████████████████████| 394.7MB 36kB/s 
     |████████████████████████████████| 3.7MB 8.7MB/s 


In [2]:
from typing import Dict, Text

import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds
#tensorflow_recommenders是Google開放的推薦引擎模型
import tensorflow_recommenders as tfrs

In [3]:
# 讀取評分資料
ratings = tfds.load('movielens/100k-ratings', split="train")
# 讀取電影相關的特徵
movies = tfds.load('movielens/100k-movies', split="train")

# 建立基礎使用者與電影資料
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"]
})
movies = movies.map(lambda x: x["movie_title"])

Dataset movielens downloaded and prepared to /root/tensorflow_datasets/movielens/100k-ratings/0.1.0. Subsequent calls will reuse this data.


Dataset movielens downloaded and prepared to /root/tensorflow_datasets/movielens/100k-movies/0.1.0. Subsequent calls will reuse this data.


In [4]:
#將使用者映射到代號
user_ids_vocabulary = tf.keras.layers.experimental.preprocessing.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(ratings.map(lambda x: x["user_id"]))
#將電影名稱映射到代號
movie_titles_vocabulary = tf.keras.layers.experimental.preprocessing.StringLookup(mask_token=None)
movie_titles_vocabulary.adapt(movies)

In [5]:
class MovieLensModel(tfrs.Model):
  def __init__(
      self,
      user_model: tf.keras.Model,
      movie_model: tf.keras.Model,
      task: tfrs.tasks.Retrieval):
    super().__init__()

    # 建立用戶與電影的內容
    self.user_model = user_model
    self.movie_model = movie_model

    # 建立檢索任務
    self.task = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # 建立計算損失的函式

    user_embeddings = self.user_model(features["user_id"])
    movie_embeddings = self.movie_model(features["movie_title"])

    return self.task(user_embeddings, movie_embeddings)

In [6]:
# 建立用戶與電影的模型
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocab_size(), 64)
])
movie_model = tf.keras.Sequential([
    movie_titles_vocabulary,
    tf.keras.layers.Embedding(movie_titles_vocabulary.vocab_size(), 64)
])

# 建立預測目標
task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
    movies.batch(128).map(movie_model)
  )
)

In [7]:
# 建立檢索模型
model = MovieLensModel(user_model, movie_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))
model.fit(ratings.batch(4096), epochs=3)

# 透過brute-force search去處理模型的結果
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index(movies.batch(100).map(model.movie_model), movies)



Epoch 1/3
25/25 [==============================] - 29s 1s/step - factorized_top_k/top_1_categorical_accuracy: 5.0000e-05 - factorized_top_k/top_5_categorical_accuracy: 0.0015 - factorized_top_k/top_10_categorical_accuracy: 0.0050 - factorized_top_k/top_50_categorical_accuracy: 0.0433 - factorized_top_k/top_100_categorical_accuracy: 0.0981 - loss: 33094.7596 - regularization_loss: 0.0000e+00 - total_loss: 33094.7596
Epoch 2/3
25/25 [==============================] - 28s 1s/step - factorized_top_k/top_1_categorical_accuracy: 2.0000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0048 - factorized_top_k/top_10_categorical_accuracy: 0.0142 - factorized_top_k/top_50_categorical_accuracy: 0.1060 - factorized_top_k/top_100_categorical_accuracy: 0.2093 - loss: 31012.0800 - regularization_loss: 0.0000e+00 - total_loss: 31012.0800
Epoch 3/3
25/25 [==============================] - 28s 1s/step - factorized_top_k/top_1_categorical_accuracy: 4.8000e-04 - factorized_top_k/top_5_categorical_accu

In [8]:
# 獲得推薦結果
_, titles = index(np.array(["33"]))
print(f"Top 3 recommendations for user 33: {titles[0, :3]}")

Top 3 recommendations for user 33: [b"Devil's Advocate, The (1997)" b'Desperate Measures (1998)'
 b'Desperate Measures (1998)']
